# DATA 608 Story 1
Title: Infrastructure Investment & Jobs Act Funding Allocation

Yina Qiao

## Intro:
In the exploration of the Infrastructure Investment and Jobs Act funding allocations, I delve into the complexities of equitable distribution and the subtle interplay of politics. The goal is to understand whether the funding reaches states and territories in a manner that reflects their population and to consider how the political landscape might influence these decisions.







### Data
I've harnessed datasets below for the analysis, carefully preprocessing them in Excel and hosting on GitHub for accessibility. Their sources in the reference section

1. Data on the present allocation of the Infrastructure Investment and Jobs Act funding by State and Territory (funding data)
2. Estimated population data for each state and territory (population data)
3. Official election results of the 2020 Presidential election (election data)

##  Preprocessing


In [6]:
import pandas as pd
import plotly.express as px
import requests
from io import BytesIO
from plotly.subplots import make_subplots
import plotly.graph_objects as go
funding_url = 'https://github.com/yinaS1234/data-608/raw/main/S1/IIJA%20FUNDING%20AS%20OF%20MARCH%202023(1).xlsx'
population_url = 'https://github.com/yinaS1234/data-608/raw/main/S1/Current%20Population%20by%20State.xlsx'
election_results_url = 'https://github.com/yinaS1234/data-608/raw/main/S1/2020_Presidential_Election_Results.xlsx'
def read_excel_from_github(url):
    response=requests.get(url)
    data=BytesIO(response.content)
    return pd.read_excel(data)
funding=read_excel_from_github(funding_url)
population=read_excel_from_github(population_url)
election=read_excel_from_github(election_results_url)

funding['State/Territory'] = funding['State/Territory'].str.title()
population['State/Territory'] = population['State/Territory'].str.title()
election['State/Territory'] = election['State/Territory'].str.title()
final_data = pd.merge(pd.merge(election, population, on='State/Territory', how='outer'), funding, on='State/Territory', how='outer' )


# Calculate per capita funding allocation (in dollars)
final_data['Per Capita Funding'] = (final_data['Total (Billions)'] * 1e9) / final_data['Population 2023']

# Replace any potential NaN values with zeros for further analysis
final_data.fillna(0, inplace=True)

# Create a column 'State_Abbrev' for easy visual.
state_abbreviations = {
    **{  # Standard states
        'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
        'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
        'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
        'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
        'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
        'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
        'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
        'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
        'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
        'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
    },
    **{  # Territories and special cases
        'District Of Columbia': 'DC', 'Puerto Rico': 'PR', 'Guam': 'GU', 'Northern Mariana Islands': 'MP',
        'US Virgin Islands': 'VI', 'American Samoa': 'AS', 'Tribal Communities': 'TC'
    }
}
state_abbreviations['Us Virgin Islands'] = 'VI'
final_data['State_Abbrev'] = final_data['State/Territory'].map(state_abbreviations)
final_data['Per Capita Funding ($1k)'] = round(final_data['Per Capita Funding'] / 1000, 0).astype(int)
final_data['Winning Party'] = final_data.apply(lambda row: 'Biden' if row['Biden Electoral Votes'] > 0 else 'Trump', axis=1)
final_data.head()

,State/Territory,Biden Electoral Votes,Trump Electoral Votes,Biden Popular Vote,Trump Popular Vote,Population 2023,Total (Billions),Per Capita Funding,State_Abbrev,Per Capita Funding ($1k),Winning Party
0,Alabama,0.0,9.0,849.0,1442.0,5108468,3.0,587.260212,AL,1,Trump
1,Alaska,0.0,3.0,153.0,189.0,733406,3.7,5044.954636,AK,5,Trump
2,Arizona,11.0,0.0,1672.0,1662.0,7431344,3.5,470.978063,AZ,0,Biden
3,Arkansas,0.0,6.0,423.0,760.0,3067732,2.8,912.726405,AR,1,Trump
4,California,55.0,0.0,11110.0,6006.0,38965193,18.4,472.216319,CA,0,Biden


## Visuals

In [7]:
fig_choropleth_funding = px.choropleth(
    final_data, 
    locations='State_Abbrev', 
    locationmode="USA-states", 
    color='Per Capita Funding ($1k)',
    scope="usa",
    color_continuous_scale="Purples"# Changed to a Teal color scale
    
)

fig_choropleth_funding.update_layout(
    title={
        'text': "Per Capita Funding by State",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    }
)




fig_choropleth_funding.show()



political_choropleth_data = final_data[['State_Abbrev', 'Winning Party']]


fig_choropleth_political = px.choropleth(
    political_choropleth_data, 
    locations='State_Abbrev', 
    locationmode="USA-states", 
    color='Winning Party',
    scope="usa",
    color_discrete_map={'Biden': 'blue', 'Trump': 'red'}
    
)
fig_choropleth_political.update_layout(
    title={
        'text': "Political Alignment by State (2020)",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24  
        )
    }
)

In [8]:


# Create a figure with 1 row and 2 columns, specifying the subplot type as 'geo'
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "geo"}, {"type": "geo"}]])

# Add the choropleth for funding to the first subplot with custom color bar position
fig.add_trace(
    go.Choropleth(
        z=final_data['Per Capita Funding ($1k)'],
        locations=final_data['State_Abbrev'],
        locationmode='USA-states',
        colorscale="Purples",
        text=final_data['State_Abbrev'] + '<br>Funding: ' + final_data['Per Capita Funding ($1k)'].astype(str) + 'k',  # Hover text
        hoverinfo='text',  
        colorbar=dict(
            title='Per CapitaFunding ($1k)',
            title_side='top',
            x=0.2,  # Adjust the position of the color bar
            y=-0.4,
            lenmode="fraction", len=0.20,  # Control the length of the color bar
            orientation="h"
            
        ),
    ),
    row=1, col=1
)

# Add the choropleth for political alignment to the second subplot with custom color bar position
fig.add_trace(
    go.Choropleth(
        z=final_data['Winning Party'].replace({'Biden': 1, 'Trump': 0}),
        locations=final_data['State_Abbrev'],
        locationmode='USA-states',
        colorscale=[(0, 'red'), (1, 'blue')],
        text=final_data['State_Abbrev'] + '<br> ' + final_data['Winning Party'],  # Hover text
        hoverinfo='text',
        colorbar=dict(
            title='Winning<br>Party',
            title_side='top',
            ticktext=['Trump', 'Biden'],
            tickvals=[0.25,0.75],
            x=0.8,  # Adjust the position of the color bar
            y=-0.4,
            lenmode="fraction", len=0.2,  # Control the length of the color bar
            orientation="h"
        ),
    ),
    row=1, col=2
)

# Update the layout for geo subplots
fig.update_geos(
    scope='usa',
    projection_type='albers usa'
)

# Update the overall layout
fig.update_layout(
    title_text="<span style='font-size:1.2em;'><b>Top </b></span><span style='font-size:1.2em;'><b>Funding</b></span> <span style='font-size:1em;'>States </span><span style='font-size:1.2em;'><b>All</b></span> <span style='font-size:1em;'>Voted for </span><span style='font-size:1.2em;'><b>Trump</b></span><br><br><span style='font-size:1.2em;'>Funding Allocation </span><span style='font-size:1.4em;'><b>Not Favor Biden</b></span>",
    title_x=0.5,
    height=400,
)

# Show the figure
fig.show()


In [9]:
t2=final_data.sort_values(by='Per Capita Funding ($1k)', ascending=False).head(2)['State/Territory'].to_list()
t2

['Alaska', 'Wyoming']

In [12]:
import plotly.graph_objects as pl
fig.add_shape(type="path",
              path="M 0.18,0.62 Q 0.5,1.1 0.74,0.62",
              line_color="Orange",
              xref="paper", yref="paper")


fig.add_shape(type="path",
              path="M 0.14,0.20 Q 0.5,0.8 0.69,0.19",
              line_color="Orange",
              xref="paper", yref="paper")


## Conclusion

Does the allocation favor the political interests of the Biden administration?


* The allocation of high per capita funding to states that voted for Trump in 2020 suggests that funding decisions do not specifically favor the political interests of the Biden administration.




Reference:

https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html

https://www.archives.gov/electoral-college/2020

https://en.wikipedia.org/wiki/2020_United_States_presidential_election